## Dependencies

In [44]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
import scipy.stats as stats
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
import sklearn as sk
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
#from ipynb.fs.full.driver_drowsiness_extraction import select_channel
import numpy as np
import pandas as pd
from scipy import stats

## Feature Selection

In [45]:
def channel_selection(features, labels, channel_list):
    '''
    Select the desired channels from the total feature dataset
    '''
    selected_channels = []
    for channel in channel_list:
        selected_channels.append(features.loc[features['channels'] == channel])
    # return the corresponding labels for the selected channels
    selected_labels = labels[0:2022*len(channel_list)].to_numpy()
    return ((pd.concat(selected_channels).drop('channels', axis=1)), selected_labels)

In [46]:
def feature_selection(selected_channels, feature_subset):
    ''' 
    Select the desired subset of features to prepare training data on.
    '''
    selected_features = pd.DataFrame()
    for feature in feature_subset:
        selected_features[feature] = selected_channels[feature]
    # temporary sanity check, will delete
    # print(selected_features.head())
    return selected_features

## Preparation and Training

In [57]:
def data_preparation(selected_channels, selected_labels, feature_subset, split_size = 0.2, seed = 1):

    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    X = feature_selection(selected_channels=selected_channels, feature_subset = feature_subset) # select every feature
    y = selected_labels

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = split_size, random_state = seed)

    # apply normalization after splitting to avoid leakage
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return [X_train, X_test, y_train, y_test]



In [58]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

def model_training(data, model_family, display_labels, stats=False, cm=False):

  X_train, X_test, y_train, y_test = data
  if model_family == 'K-NN':
    model = KNeighborsClassifier()
  elif model_family == 'DTC':
    model = DecisionTreeClassifier()
  elif model_family == 'RFC':
    model = RandomForestClassifier(n_estimators=100)
  elif model_family == 'Logistic Regression':
    model = LogisticRegression(max_iter=5000)
  elif model_family == 'SVM':
    model = SVC(C=1.0, kernel='rbf', degree=10, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=1)
  elif model_family == 'NN':
    model = MLPClassifier(activation='relu',solver='adam', alpha=1e-2, learning_rate='adaptive', max_iter=1000000, hidden_layer_sizes=(60,2), random_state=1)
  elif model_family == 'GBC':
    model = GradientBoostingClassifier(loss='log_loss',n_estimators=300, learning_rate=0.1, max_depth=10, random_state=1)

  model.fit(X_train, y_train)
  print('Accuracy of {} classifier on training set: {:.8f}'
     .format(model_family, model.score(X_train, y_train)))
  print('Accuracy of {} classifier on test set: {:.8f}'
     .format(model_family, model.score(X_test, y_test)))

  if stats:
    print()
    print("==== Stats for the {} model ====".format(model_family))
    sensitivity = recall_score(y_test, model.predict(X_test))
    print("Sensitivity (Recall):", sensitivity)

    precision = precision_score(y_test, model.predict(X_test))
    print("Precision:", precision)

    accuracy = accuracy_score(y_test, model.predict(X_test))
    print("Accuracy (Recall):", accuracy)
        
    f1 = f1_score(y_test, model.predict(X_test))
    print("F1_score:", f1)

    fpr, tpr, thresholds = roc_curve(y_test, model.predict(X_test))
    auc = roc_auc_score(y_test, model.predict(X_test))
    print("AUC:", auc)

    logloss = log_loss(y_test, model.predict(X_test))
    print("Logloss:", logloss)
    print()

  if cm:
    model_cm = confusion_matrix(y_test, model.predict(X_test))
    model_disp = ConfusionMatrixDisplay(confusion_matrix=model_cm,display_labels=display_labels)
    model_disp.plot()

## I/O

In [59]:
# load the feature dataset as a dataframe
csv_file = 'eeg_features.csv'
df = pd.read_csv(csv_file,float_precision='round_trip')
df = df.drop('Unnamed: 0', axis=1)

In [60]:
# split the dataset to features and labels
features = df.drop('label', axis=1)
labels = df.iloc[:,-1:]
display_labels = ['drowsy' if label == 1 else 'alert' for label in labels['label'].unique()]

## Testing and Processing

In [61]:
# select the channels to be processed
channel_list = ['F3', 'F4','C3','Cz','Oz']

# select the models to be trained
models = ['GBC']

In [62]:
selected_channels, selected_labels = channel_selection(features=features, labels=labels, channel_list=channel_list)
data = data_preparation(selected_channels=selected_channels, selected_labels=selected_labels, feature_subset=selected_channels.columns)

for model in models:
    model_training(data, model, display_labels, stats=True, cm=False)

Accuracy of GBC classifier on training set: 1.00000000
Accuracy of GBC classifier on test set: 0.80316518

==== Stats for the GBC model ====
Sensitivity (Recall): 0.7810650887573964
Precision: 0.8181818181818182
Accuracy (Recall): 0.8031651829871415
F1_score: 0.7991927346115035
AUC: 0.8032309570771109
Logloss: 7.094645919321774



## Evaluation

## Archive

In [55]:
# TODO: select different feature subsets, then apply the p-value thresholding to see if there's any inconsistencies
# after that we can either implement the p values as a function to select the top performers of a subset (subset of the subset)
# or we can just select our own features and do the processing on them

# if we're going to use multiple subsets then write another function definition for the preparation of X_train and Y_train etc.
# the function should return x_train y_train x_test y_test and it should take the selected feature subset as the input
# after that we can run the selections in their respective cells to see the performances of different models
# this can also allow us to compare the model performances with respect to feature selection which could make a great performance matrix

# P-Value Thresholding for Feature Selection
def p_value_thresholding(selected_features, selected_labels):

    p_values = []

    #X_p = selected_features.drop('channels',axis = 1)
    #X_p = X_p.drop('label',axis = 1)
    X_p = selected_features
    
    y_p = selected_labels.flatten()
    y_p = pd.Series(y_p)


    #y_p = pd.Series(y['0'])
    for feature in X_p.columns:
        t_stat, p_value = stats.ttest_ind(X_p[feature][y_p == 0], X_p[feature][y_p == 1])
        p_values.append(p_value)

    alpha = 0.05

    # Select features with p-values below the significance level
    selected_features = [X_p.columns[i] for i, p in enumerate(p_values) if p < alpha]
    # Alternatively, you can rank features by p-value
    sorted_features = [x for _, x in sorted(zip(p_values, X_p.columns))]
    return sorted_features

In [56]:
all = feature_selection(selected_channels=selected_channels, feature_subset=selected_channels.columns) # select every feature
p_all = p_value_thresholding(selected_features=all, selected_labels=y)

spc = feature_selection(selected_channels=selected_channels, feature_subset=['gamma_beta', 'alpha_power', 'delta_power', 'beta_power', 'theta_power', 'zc'])
p_spc = p_value_thresholding(selected_features=spc, selected_labels=y)
print(p_spc)

['zc', 'gamma_beta', 'alpha_power', 'beta_power', 'theta_power', 'delta_power']


In [25]:
p_all

['spc_roff',
 'mfcc_0',
 'spc_cnt',
 'mfcc_1',
 'mfcc_2',
 'zc',
 'dfa',
 'gamma_beta',
 'mfcc_3',
 'mel_4',
 'mel_7',
 'chr_3',
 'chr_2',
 'chr_7',
 'mel_5',
 'chr_4',
 'chr_5',
 'mel_6',
 'chr_1',
 'chr_6',
 'mel_3',
 'mel_0',
 'mel_1',
 'mel_2',
 'gamma_alpha',
 'chr_0',
 'chr_8',
 'mel_8',
 'alpha_delta',
 'mel_9',
 'alpha_theta',
 'chr_14',
 'beta_alpha',
 'chr_9',
 'mfcc_4',
 'gamma_delta',
 'gamma_theta',
 'chr_10',
 'beta_theta',
 'chr_11',
 'theta_delta',
 'beta_delta',
 'chr_13',
 'chr_12',
 'alpha_power',
 'gamma_power',
 'beta_power',
 'theta_power',
 'delta_power',
 'mean_abs_sec_dif']

In [ ]:
'''
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler

#y = y.reset_index(drop=True)

pca = PCA(n_components = 0.999)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
#X = dataPCA
variance = pd.DataFrame(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_)
print(np.sum(pca.explained_variance_ratio_))
'''